In [17]:
import numpy as np
import pandas as pd
import pcg_skel
from caveclient import CAVEclient
client = CAVEclient('minnie65_phase3_v1')

In [ ]:
import re
import html2text
def handle_error_message(error_message):
    if not isinstance(error_message, str):
        return str(error_message)
    prefix_match = re.match(r"(\d{3} .+?) Server Error for url: (.+?) content:", error_message)
    prefix = prefix_match.group(1) if prefix_match else ""
    endpoint_error = prefix_match.group(2) if prefix_match else ""

    html_content = re.search(r"content:b'(.*?)'$", error_message, re.DOTALL)
    if html_content:
        converter = html2text.HTML2Text()
        converter.ignore_links = True
        converter.ignore_images = True
        converter.ignore_newlines = True
        converter.ignore_emphasis = True
        content = converter.handle(html_content.group(1))
        content = content.strip()

        if prefix:
            return f"{prefix} at URL: {endpoint_error}"
        return content

    return error_message


In [63]:
error_message = "503 Server Error: Service Temporarily Unavailable for url: https://minnie.microns-daf.com/l2cache/api/v1/table_mapping content:b'<html>\r\n<head><title>503 Service Temporarily Unavailable</title></head>\r\n<body>\r\n<center><h1>503 Service Temporarily Unavailable</h1></center>\r\n<hr><center>nginx</center>\r\n</body>\r\n</html>\r\n'"

In [64]:
handle_error_message(error_message)

'503 Server Error: Service Temporarily Unavailable at URL: https://minnie.microns-daf.com/l2cache/api/v1/table_mapping'

In [8]:
client.info.get_image_mirror_names()

['Minnie65 Google', 'Minnie65 BossDB', 'Minnie65 Asia']

In [15]:
client.info.get_datastack_info(image_mirror=None)

{'aligned_volume': {'id': 1,
  'name': 'minnie65_phase3',
  'display_name': 'Minnie65',
  'description': "This is the second alignment of the IARPA 'minnie65' dataset, completed in the spring of 2020 that used the seamless approach.",
  'image_source': 'precomputed://https://storage.googleapis.com/iarpa_microns/minnie/minnie65/em'},
 'segmentation_source': 'graphene://https://minnie.microns-daf.com/segmentation/table/minnie3_v1',
 'skeleton_source': 'precomputed://middleauth+https://minnie.microns-daf.com/skeletoncache/api/v1/minnie65_phase3_v1/precomputed/skeleton',
 'analysis_database': None,
 'viewer_site': 'https://neuroglancer.neuvue.io/',
 'synapse_table': 'synapses_pni_2',
 'soma_table': 'nucleus_neuron_svm',
 'local_server': 'https://minnie.microns-daf.com',
 'description': 'This is the first version of Minnie that has proofreading enabled. Was first enabled on June 24, 2020. ',
 'viewer_resolution_x': 4.0,
 'viewer_resolution_y': 4.0,
 'viewer_resolution_z': 40.0,
 'proofreadi

In [16]:
client.info.get_datastack_info()

{'aligned_volume': {'id': 1,
  'name': 'minnie65_phase3',
  'display_name': 'Minnie65',
  'description': "This is the second alignment of the IARPA 'minnie65' dataset, completed in the spring of 2020 that used the seamless approach.",
  'image_source': 'precomputed://https://storage.googleapis.com/iarpa_microns/minnie/minnie65/em'},
 'segmentation_source': 'graphene://https://minnie.microns-daf.com/segmentation/table/minnie3_v1',
 'skeleton_source': 'precomputed://middleauth+https://minnie.microns-daf.com/skeletoncache/api/v1/minnie65_phase3_v1/precomputed/skeleton',
 'analysis_database': None,
 'viewer_site': 'https://neuroglancer.neuvue.io/',
 'synapse_table': 'synapses_pni_2',
 'soma_table': 'nucleus_neuron_svm',
 'local_server': 'https://minnie.microns-daf.com',
 'description': 'This is the first version of Minnie that has proofreading enabled. Was first enabled on June 24, 2020. ',
 'viewer_resolution_x': 4.0,
 'viewer_resolution_y': 4.0,
 'viewer_resolution_z': 40.0,
 'proofreadi

In [ ]:
from tourguide.flask_app.api import get_image_mirrors

In [3]:
client.info.get_image_mirror_names(datastack_name="minnie65_phase3_v1")

['Minnie65 Google', 'Minnie65 BossDB', 'Minnie65 Asia']

In [3]:
import datetime

In [5]:
root_id = 864691135891272841
now = datetime.datetime.now(datetime.timezone.utc)
curr_ids = client.chunkedgraph.get_latest_roots(root_id, timestamp=now)

In [8]:
curr_ids[client.chunkedgraph.is_latest_roots(curr_ids)]

array([864691135977354819, 864691136813834611])

In [2]:
from guidebook.guidebook_lib import processing, states
from guidebook.guidebook_app import utils

states.set_default_ngl_configuration(
    client.datastack_name,
    target_site = 'spelunker',
    target_url = None,
)

In [3]:
root_id = 864691135891272841
split_point = [175417, 186193, 19949]

In [4]:
nrn = pcg_skel.get_meshwork_from_client(root_id = root_id, client=client)


In [5]:
vertex_df = processing.process_meshwork_to_dataframe(nrn)

In [ ]:
path_df, l2_ids = processing.process_paths(
    root_id,
    vertex_df,
    client=client,
    compartment_filter='axon',
    return_l2_ids=True,
    min_path_length=10_000,
    step_size=2_000,
)

sb_data = states.package_path_data(
    path_df,
    l2_ids,
    client=client,
)

In [9]:
sb = states.make_path_statebuilder(
    color='white',
    resolution=[4,4,40],
    use_skeleton_service=True,
    split_positions=True,
    client=client,
    root_ids=[root_id],
    add_restricted_segmentation_layer=True,
    restricted_color='turquoise',
)

In [ ]:
print(
    sb.render_state(sb_data, return_as='shorten')
)

In [ ]:
min_cover

In [ ]:
sb = states.make_point_statebuilder(
    ['ends', 'branches'],
    resolution=[4,4,40],
    use_skeleton_service=True,
    split_positions=True,
    client=client,
    root_ids=[root_id],
)

In [10]:
sb = states.make_path_statebuilder(
    color='white',
    resolution=[4,4,40],
    use_skeleton_service=True,
    split_positions=True,
    client=client,
    root_ids=[root_id],
)

In [ ]:
print(
    sb.render_state({'cover-path': path_df}, return_as='short', config_key=client.datastack_name)
)

In [ ]:
np.linalg.norm(path_df[['pointA_x', 'pointA_y', 'pointA_z']].values - path_df[['pointB_x', 'pointB_y', 'pointB_z']].values, axis=1)

In [ ]:
np.array([np.nan, np.nan, np.nan]).reshape((1,3))

In [12]:
sk = nrn.skeleton

In [ ]:
paths = sk.cover_paths_with_parent()

In [ ]:
np.array(sk.path_length(paths=paths))

In [ ]:
from scipy import interpolate

In [37]:
ds = sk.distance_to_root[cps[0][::-1]]

In [38]:
step_size = 1_000

In [56]:
def interpolate_path(path, sk, step_size):
    path = path[::-1]
    ds = sk.distance_to_root[path]
    new_ds = np.arange(ds[0], ds[-1], step_size)
    return np.vstack(
        [
            np.interp(new_ds, ds, sk.vertices[path, 0]),
            np.interp(new_ds, ds, sk.vertices[path, 1]),
            np.interp(new_ds, ds, sk.vertices[path, 2]),
        ],
    ).T

In [ ]:
interpolate_path(cps[0], sk, 1_000)

In [ ]:
ds_interp = np.linspace(
    ds[0], ds[-1], np.round((ds[-1] - ds[]) / step_size).astype(int),
)

In [41]:
xs = nrn.skeleton.vertices[cps[0][::-1], 0]

In [ ]:
np.interp(
    ds_interp,
    ds,
    xs,
)

In [ ]:
colnames = [f"{pre}_{suf}" for pre in ['pointA', 'pointB'] for suf in ['x', 'y', 'z'] ]

pd.DataFrame(
    np.concatenate([sk.vertices[cps[0][:-1]], sk.vertices[cps[0][1:]]], axis=1),
    columns=colnames,
)

In [8]:
split_lvl2_id = pcg_skel.chunk_tools.get_closest_lvl2_chunk(
    point=split_point,
    root_id=root_id,
    client=client,
    voxel_resolution=client.info.viewer_resolution(),
    radius=300,
)


In [ ]:
processing.add_downstream_column(
    vertex_df,
    split_lvl2_id,
).query('is_downstream')

In [6]:
l2s_long = vertex_df['lvl2_id'].explode()
split_vert = l2s_long[l2s_long==split_lvl2_id].index[0]

In [ ]:
l2s_long = vertex_df['lvl2_id'].explode()
split_vert = l2s_long[l2s_long==split_lvl2_id].index[0]

In [16]:
from meshparty import skeleton

In [ ]:
vertex_df.head()

In [ ]:
vertex_df.query(f"{"parent"}!=-1")["parent"].reset_index().values

In [ ]:
skeleton.Skeleton(
    vertices=vertex_df[['pt_x', 'pt_y', 'pt_z']].values,
    
)

In [ ]:
vertex_df_filt = processing.filter_dataframe(
    root_id,
    vertex_df,
    restriction_direction='downstream-of',
    restriction_point=[175417, 186193, 19949],
    client=client,
)

In [ ]:
vertex_df

In [6]:
from datetime import datetime, timedelta, UTC

In [ ]:
vertex_df = processing.add_timestamp_relative_vertex(
    root_id=root_id,
    vertex_df=vertex_df,
    client=client,
    horizon_timestamp=datetime.now(UTC) - timedelta(days=365)
)

In [ ]:
vertex_df.query('not_in_previous_match')

In [ ]:
vertex_df['lvl2_id'].explode()

In [10]:
l2all = vertex_df['lvl2_id'].explode()

In [7]:
ts = client.chunkedgraph.get_root_timestamps(
    vertex_df['lvl2_id'].explode()
)

In [16]:
ts_df = pd.DataFrame(
    {
        'lvl2_id': l2all,
        'timestamp': [t.timestamp() for t in ts],
    }
)

In [ ]:
client.chunkedgraph.get_past_ids?

In [ ]:
other_roots, frac_overlap = client.chunkedgraph.suggest_latest_roots(
    root_id=864691135891272841,
    timestamp=datetime.now(UTC) - timedelta(days=365),
    return_all=True,
    stop_layer=2,
    return_fraction_overlap=True,
)

In [60]:
base_lvl2 = client.chunkedgraph.get_leaves(864691135891272841, stop_layer=2)

In [62]:
other_lvl2 = client.chunkedgraph.get_leaves(864691135214134328, stop_layer=2)

In [ ]:
len(np.intersect1d(base_lvl2, other_lvl2)) / len(base_lvl2)

In [85]:
def get_highest_overlap_lvl2_ids(root_id, timestamp, client):
    id_dict = client.chunkedgraph.get_past_ids(
        root_ids=root_id,
        timestamp_past=timestamp,
    )
    base_lvl2 = client.chunkedgraph.get_leaves(root_id, stop_layer=2)
    past_ids = np.array(id_dict['past_id_map'].get(root_id, []))
    past_ids = past_ids[np.argsort(client.chunkedgraph.get_root_timestamps(past_ids))[::-1]]
    relative_fracs = []
    past_lvl2 = {}
    for rid in past_ids:
        past_lvl2[rid] = client.chunkedgraph.get_leaves(rid, stop_layer=2)
        rel_frac = len(np.intersect1d(base_lvl2, past_lvl2[rid])) / len(base_lvl2)
        relative_fracs.append(rel_frac)
        if max(relative_fracs) > 1-sum(relative_fracs):
            break
    highest_past = past_ids[np.argmax(relative_fracs)]
    return past_lvl2[highest_past]

In [ ]:
np.argsort(client.chunkedgraph.get_root_timestamps(other_roots))

In [87]:
match_l2s = get_highest_overlap_lvl2_ids(864691135891272841, datetime.now(UTC) - timedelta(days=365), client)

In [93]:
existed_before = vertex_df['lvl2_id'].explode().isin(match_l2s)
existed_before[existed_before].index.unique()

In [ ]:
client.chunkedgraph.get_past_ids(
    root_ids=864691135891272841,
    timestamp_past=datetime.now(UTC) - timedelta(days=365),
)['past_id_map'][864691135891272841]

In [ ]:
ts_df.

In [ ]:
utils.update_seen_id_list([], vertex_df).tolist()

In [ ]:
seen_svids = utils.stash_id_list(utils.update_seen_id_list([], vertex_df))

In [ ]:
utils.stash_id_list(utils.update_seen_id_list(seen_svids, vertex_df))

In [ ]:
l2seen = vertex_df['lvl2_id'].explode().isin(seen_svids)
l2seen_idx = l2seen[l2seen].index.unique()

In [ ]:
l2seen_idx

In [8]:
l2_ids = [123, 421, 5445]
new_ids = [123, 111, 59593]

In [ ]:
utils.rehydrate_id_list(
    utils.stash_id_list(l2_ids)
)

In [ ]:
utils.update_seen_id_list(l2_ids, pd.DataFrame({'lvl2_id': new_ids}))

In [ ]:
client.chunkedgraph.suggest_latest_roots(864691136723961341)

In [5]:
syn_df = client.materialize.synapse_query(pre_ids=864691135891272841)

In [ ]:
syn_df['new_thing'] = syn_df['id'].isin([151698981, 168321387])

In [ ]:
syn_df.new_thing

In [ ]:
lvl2_ids = client.chunkedgraph.get_leaves(864691136723961341, stop_layer=2)

In [ ]:
client.chunkedgraph.get_root_timestamps(lvl2_ids)

In [ ]:
client.chunkedgraph.get_root_timestamps(86469113589127284)

In [ ]:
ts

In [ ]:
syn_df['size'].nunique()

In [ ]:
syn_df.id.tolist()

In [12]:
nsyn_df = client.materialize.query_table(
    'synapses_pni_2',
    filter_in_dict={'id': syn_df.id.tolist()}
)

In [3]:
root_id = 864691136723961341
nrn = pcg_skel.service.get_meshwork_from_client(
    root_id=root_id,
    client=client,
)

In [4]:
df = processing.process_meshwork_to_dataframe(nrn)

In [ ]:
df

In [ ]:
processing.add_downstream_column(df, base_lvl2_id=157572592595632752).value_counts('is_downstream')

In [ ]:
dfe = 

In [16]:
base_id = 156868424184627514

In [23]:
first_skind = df.explode('lvl2_id').query('lvl2_id==@base_id')['parent'].values[0]

In [26]:
upstream_skind = []
parents = df['parent'].values
next_parent = first_skind
while next_parent >= 0:
    upstream_skind.append(int(next_parent))
    next_parent = parents[next_parent]

In [28]:
from scipy import sparse

In [29]:
from meshparty import skeleton

In [ ]:
df

In [ ]:
df.query('parent!=-1')['parent'].reset_index().values

In [ ]:
int(df.query('is_root').index[0])

In [48]:
newsk = skeleton.Skeleton(
    vertices=df[['x', 'y', 'z']].values,
    edges=df.query('parent!=-1')['parent'].reset_index().values,
    root=int(df.query('is_root').index[0])
)

In [49]:
first_skind = df.explode('lvl2_id').query('lvl2_id==@base_id')['parent'].values[0]

In [ ]:
newsk.downstream_nodes(first_skind)

In [ ]:
andvert_df["is_end_point"] = False
vert_df.loc[nrn.skeleton.end_points, "is_end_point"] = True

vert_df["is_branch_point"] = False
vert_df.loc[nrn.skeleton.branch_points, "is_branch_point"] = True


In [ ]:
vert_df.loc[4434, 'is_root'] = True

In [ ]:
vert_df["is_root"] = False
vert_df.loc[int(nrn.skeleton.root), "is_root"] = True

In [ ]:
v
ert_df["is_end_point"] = False
vert_df.loc[nrn.skeleton.end_points, "is_end_point"] = True

vert_df["is_branch_point"] = False
vert_df.loc[nrn.skeleton.branch_points, "is_branch_point"] = True

vert_df["is_root"] = False
vert_df.loc[nrn.skeleton.root, "is_root"] = True

In [ ]:
processing.process_meshwork_to_dataframe(nrn)

In [ ]:
obj

In [6]:
df = client.skeleton.get_skeleton(root_id, output_format='swc')

In [13]:
verts = nrn.skeleton.vertices
is_axon = nrn.anno.is_axon.mesh_index.to_skel_mask
edges = nrn.skeleton.edges

In [21]:
import pandas as pd
vert_df = pd.DataFrame(
    {
        'x': verts[:,0].astype(int),
        'y': verts[:,1].astype(int),
        'z': verts[:,2].astype(int),
        'is_axon': is_axon,
    }
)

vert_df['is_end_point'] = False
vert_df.loc[nrn.skeleton.end_points, 'is_end_point'] = True


In [35]:
l2df = nrn.anno.lvl2_ids.df
l2df['skind'] = nrn.skeleton.mesh_to_skel_map
l2df.groupby('skind')['lvl2_id'].agg(list)

In [ ]:
nrn.skeleton_indices

In [ ]:
sk.parent_nodes(nrn.skeleton_indices)

In [2]:
root_id = 864691136723961341
nrn = pcg_skel.service.get_meshwork_from_client(
    root_id=root_id,
    client=client,
    synapses=True,
)

In [ ]:
nrn.anno.is_axon.mesh_index.to_skel_mask

In [58]:
sk = client.skeleton.get_skeleton(root_id, skeleton_version=4)

In [ ]:
np.array(sk['compartment'])

In [ ]:
m2s_map = np.array(sk['mesh_to_skel_map'])

In [69]:
compartments = np.array(sk['compartment'])

In [ ]:
compartments[m2s_map]

In [ ]:
len(sk['mesh_to_skel_map'])

In [ ]:
nrn.anno.compartment.df.value_counts('compartment')

In [55]:
comp_mask = nrn.MeshIndex(nrn.anno.compartment.df.query('compartment==2').mesh_ind.values).to_mesh_mask

In [ ]:
with nrn.mask_context(comp_mask):
    print(len(nrn.anno.pre_syn.df))
    print(len(nrn.anno.post_syn.df))

In [ ]:
client.chunkedgraph.is_valid_nodes(864691135428878637)

In [ ]:
root_id = 864691135428878637 

nrn = pcg_skel.service.get_meshwork_from_client?